# Goal:

For the sales, I'd like to denote whether it was sold at a loss or a gain with a red or green icon. The rest of the info could be put into a pop up box, if that works

oh and btw maybe put listings in blue or something so we can see that they are not sold yet

# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup
import gspread

KeyboardInterrupt: 

## Data read-in

In [ ]:
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
# import re

# # Define scope and credentials
# scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
# creds = ServiceAccountCredentials.from_json_keyfile_name("autoscraper-380600-0d0c84856d6b.json", scope)
# client = gspread.authorize(creds)

# # Open the Google Sheet
# sheet = client.open("BrandonMillerFORMAP").sheet1  # Adjust sheet1 if using different tabs

# # Get a range of cells (adjust range as needed)
# cell_range = sheet.range('A1:D13')  # Change to match your sheet range

# # Get the headers and the actual data range (adjust as necessary)
# headers = sheet.row_values(1)  # Fetch first row for headers
# data_range = sheet.get_all_values()[1:]  # Skip header row

# # Extract hyperlinks and clean data
# data = []

# def extract_hyperlink_info(formula):
#     """Extracts URL and display text from a Google Sheets HYPERLINK formula."""
#     if not formula:
#         return None, None  # Return None if there's no formula
#     pattern = r'HYPERLINK\("(.+?)",\s*"(.+?)"\)'
#     match = re.search(pattern, formula)
#     if match:
#         return match.group(2), match.group(1)  # return (display_text, url)
#     return None, None

# # Loop through rows and process the data
# for row in sheet.get_all_records():
#     # Extract values
#     address = row['Address']  # Adjust these keys if different
#     description = row['Description']
#     neighborhood = row['Neighborhood']

#     # Check if address or description contains hyperlinks
#     address_formula = sheet.cell(row['Row'], 1, value_render_option='FORMULA').value  # Address column
#     description_formula = sheet.cell(row['Row'], 2, value_render_option='FORMULA').value  # Description column
    
#     # Extract hyperlink from Address and Description columns (if present)
#     address_display, address_url = extract_hyperlink_info(address_formula)
#     description_display, description_url = extract_hyperlink_info(description_formula)
    
#     # Combine hyperlink into the display text
#     if address_url:
#         address = f'<a href="{address_url}">{address_display}</a>'
#     if description_url:
#         description = f'<a href="{description_url}">{description_display}</a>'
    
#     # Append the cleaned row to data
#     data.append([address, description, neighborhood])

# # Convert to DataFrame
# df = pd.DataFrame(data, columns=headers)

# # Display the dataframe to ensure data integrity
# print(df)

In [ ]:
df = pd.read_csv('BrandonMillerFORMAP - Sheet1.csv')

In [ ]:
def create_hyperlink_limited(description, url, word_limit=5):
    """Create an HTML anchor tag for the first few words of the description with the URL."""
    if pd.notnull(url):  # Check if there's a valid URL
        words = description.split()  # Split the description into words
        if len(words) > word_limit:  # Only wrap the first word_limit words
            first_part = ' '.join(words[:word_limit])
            remaining_part = ' '.join(words[word_limit:])
            return f'<a href="{url}">{first_part}</a> {remaining_part}'
        else:
            return f'<a href="{url}">{description}</a>'  # If fewer than word_limit words, wrap the whole text
    return description  # Return original description if no URL

# Apply the function to combine 'Description' and 'URL'
df['Description'] = df.apply(lambda row: create_hyperlink_limited(row['Description'], row['URL'], word_limit=5), axis=1)

In [ ]:
# Drop the 'URL' column if you no longer need it
df = df.drop(columns=['URL'])

In [ ]:
df['Description'].iloc[0]

## Geocode

In [ ]:
df

In [ ]:
df['full_address'] = df['Address'] + ' ' + df['Neighborhood'] + ' ' + df['Sta']

In [ ]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [ ]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'Full_Address' column and store the results in the 'geocoded' column
df['geocoded'] = df['full_address'].apply(geocode)

# Split 'geocoded' into 'lat' and 'lon', handling None values
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x is None else x).apply(pd.Series)

# Convert 'lat' and 'lon' to floats
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

# Create the 'geometry' column with Point objects
df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lat']) and pd.notnull(row['lon']) else None, axis=1)

# Drop rows where geometry is None
df = df.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set CRS (Coordinate Reference System) to WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)


In [ ]:
gdf

In [ ]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [ ]:
gdf.to_file('data.geojson', driver='GeoJSON')

In [ ]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

<a href="https://therealdeal.com/chicago/2024/04/03/david-gassman-sells-chicago-apartment-portfolio-for-72m/"> 6954 North Sheridan Road, Chicago</a>

In [ ]:
gdf.columns